In [6]:
# Install google earth engine python API
# https://developers.google.com/earth-engine/python_install_manual
%pip install earthengine-api --upgrade pandas geopandas --quiet
%pip install requests folium osmnx --quiet
%pip install matplotlib --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
%conda --version
%pip --version

conda 23.1.0

Note: you may need to restart the kernel to use updated packages.
pip 23.3.1 from /Users/hermanmitish/private/satellite-analysis/.conda/lib/python3.11/site-packages/pip (python 3.11)
Note: you may need to restart the kernel to use updated packages.


In [2]:
city_name = "Sviatohirsk"

In [6]:
import osmnx as ox
import geopandas as gpd
import os

# Make ./data directory if it doesn't exist
if not os.path.exists('./data'):
    os.makedirs('./data')

def download_buildings_and_convert_to_geojson(city_name, output_filename):
    """
    Download all buildings from a specified city using OSM and convert them to GeoJSON.
    
    Args:
    - city_name: The name of the city to download buildings from.
    - output_filename: The filename for the output GeoJSON file.
    """
    
    # Configure osmnx to retrieve buildings only
    ox.settings.use_cache=True
    ox.settings.log_console=True
    
    # Retrieve buildings from the specified city
    buildings_gdf = ox.features_from_place(city_name, tags={'building': True})
    
    # Convert the GeoDataFrame to GeoJSON format
    buildings_geojson = buildings_gdf.to_json()
    
    # Save the GeoJSON to a file
    with open(output_filename, 'w') as f:
        f.write(buildings_geojson)
    
    print(f"GeoJSON file has been saved to {output_filename}")

download_buildings_and_convert_to_geojson(city_name, f'./data/{city_name}_osm_buildings.geojson')


GeoJSON file has been saved to ./data/Sviatohirsk_osm_buildings.geojson


In [9]:
# Get the city boundary and save it to a GeoJSON file
city_boundary = ox.geocode_to_gdf(city_name)
city_boundary.to_file(f'./data/{city_name}_boundary.geojson', driver='GeoJSON')

In [11]:
# plot both buildings and boundary on the same map
import folium
import json

# Load the GeoJSON files
with open(f'./data/{city_name}_osm_buildings.geojson') as f:
    buildings_geojson = json.load(f)

with open(f'./data/{city_name}_boundary.geojson') as f:
    boundary_geojson = json.load(f)

# Create a map
m = folium.Map(location=[city_boundary['geometry'].centroid.y, city_boundary['geometry'].centroid.x], zoom_start=13)

# Add the boundary to the map
folium.GeoJson(boundary_geojson, name='Boundary').add_to(m)

# Add the buildings to the map
folium.GeoJson(buildings_geojson, name='Buildings').add_to(m)

# Display the map
m

/var/folders/yq/0zz796hs77l_70crv1b3x6z80000gn/T/ipykernel_39764/2208756844.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  m = folium.Map(location=[city_boundary['geometry'].centroid.y, city_boundary['geometry'].centroid.x], zoom_start=13)
